In [1]:
import pandas as pd

# Load the datasets with only the required columns
df1 = pd.read_csv('C:/Users/Guest01/Documents/dataset_anime/archive/final_animedataset.csv', usecols=['user_id', 'anime_id', 'my_score', 'score', 'scored_by', 'gender', 'popularity'])
df2 = pd.read_csv('C:/Users/Guest01/Documents/dataset_anime/archive/users-score-2023.csv', usecols=['user_id', 'anime_id', 'Anime Title', 'rating'])

# Merge the datasets on user_id and anime_id
final_df = pd.merge(df2, df1, on=['user_id', 'anime_id'])

# Display the first few rows to check the data
print(final_df.head())


   user_id  anime_id             Anime Title  rating  my_score gender  score  \
0        1        21               One Piece       9         9   Male   8.54   
1        1        48             .hack//Sign       7         7   Male   7.09   
2        1       320                  A Kite       5         5   Male   6.66   
3        1        49        Aa! Megami-sama!       8         8   Male   7.38   
4        1       304  Aa! Megami-sama! Movie       8         8   Male   7.63   

   scored_by  popularity  
0     423868          35  
1      61485         650  
2      18934        1946  
3      20930        1807  
4      18571        2007  


In [2]:
# Frequency encoding for categorical columns
categorical_columns = ['gender', 'Anime Title']

for col in categorical_columns:
    freq = final_df[col].value_counts()
    final_df[col] = final_df[col].map(freq)

# Handle missing values by filling with median for numerical columns
numerical_columns = ['my_score', 'score', 'scored_by', 'popularity']
for col in numerical_columns:
    final_df[col].fillna(final_df[col].median(), inplace=True)

# Display the first few rows after preprocessing
print(final_df.head())


C:\Users\Guest01\AppData\Local\Temp\ipykernel_11536\4112127917.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  final_df[col].fillna(final_df[col].median(), inplace=True)
C:\Users\Guest01\AppData\Local\Temp\ipykernel_11536\4112127917.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as 

   user_id  anime_id  Anime Title  rating  my_score   gender  score  \
0        1        21        10462       9         9  4175028   8.54   
1        1        48         5341       7         7  4175028   7.09   
2        1       320         1892       5         5  4175028   6.66   
3        1        49         2448       8         8  4175028   7.38   
4        1       304         2395       8         8  4175028   7.63   

   scored_by  popularity  
0     423868          35  
1      61485         650  
2      18934        1946  
3      20930        1807  
4      18571        2007  


C:\Users\Guest01\AppData\Local\Temp\ipykernel_11536\4112127917.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  final_df[col].fillna(final_df[col].median(), inplace=True)
C:\Users\Guest01\AppData\Local\Temp\ipykernel_11536\4112127917.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as 

In [3]:
# Define features (X) and target (y)
X = final_df.drop(columns=['rating'])  # All columns except 'rating' are features
y = final_df['rating']  # 'rating' is the target variable

# Display the shapes of X and y to confirm
print(f'X shape: {X.shape}, y shape: {y.shape}')


X shape: (6021419, 8), y shape: (6021419,)


In [4]:
from sklearn.model_selection import train_test_split

# Function to sample data
def sample_data(X, y, sample_size):
    if isinstance(sample_size, float):
        if 0 < sample_size < 1.0:
            return train_test_split(X, y, test_size=0.2, train_size=sample_size, random_state=42)
        elif sample_size == 1.0:
            return train_test_split(X, y, test_size=0.001, random_state=42)
        else:
            raise ValueError("sample_size as float must be in the range (0.0, 1.0) or equal to 1.0.")
    elif isinstance(sample_size, int):
        if sample_size > len(X):
            raise ValueError(f"sample_size {sample_size} exceeds the number of available samples {len(X)}.")
        sampled_X = X.sample(n=sample_size, random_state=42)
        sampled_y = y.loc[sampled_X.index]
        return train_test_split(sampled_X, sampled_y, test_size=0.2, random_state=42)
    else:
        raise ValueError("sample_size must be a float or an integer")


In [5]:
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score
import time
import psutil
import numpy as np

# Function to calculate and return metrics for Elastic Net with RandomizedSearchCV
def calculate_metrics(X_train, X_test, y_train, y_test):
    en = ElasticNet()

    # Define hyperparameters for RandomizedSearchCV
    # param_distributions = {
    #     'alpha': [0.01, 0.1, 0.5, 1, 2, 5, 10],  # Regularization strength
    #     'l1_ratio': np.linspace(0, 1, 20),  # Balance between L1 and L2 regularization
    #     'max_iter': [1000, 2000, 3000],  # Number of iterations
    #     'tol': [1e-4, 1e-3, 1e-2]  # Tolerance for stopping criteria
    # }
    
#     param_distributions = {
#     'alpha': np.linspace(0.01, 50, 100),  # 100 evenly spaced values between 0.01 and 50
#     'l1_ratio': np.linspace(0, 1, 50),  # 50 values between 0 and 1
#     'max_iter': [1000, 2000, 3000, 5000, 10000],
#     'tol': [1e-5, 1e-4, 1e-3]
# }

    param_distributions = {
    'alpha': np.linspace(0.01, 2, 200),  # 200 values between 0.0001 and 100 on a log scale for more granularity
    'l1_ratio': np.linspace(0, 1, 100 ),  # 100 values between 0 and 1 to finely tune the L1/L2 balance
    'max_iter': [1000, 3000, 5000, 7000, 10000, 15000],  # Increased range and number of iterations
    'tol': [1e-6, 1e-5, 1e-4, 1e-3, 1e-2],  # Adding more tolerance values for stricter convergence criteria
    'fit_intercept': [True, False], # Whether to calculate the intercept for this model
}

    random_search = RandomizedSearchCV(
        en, 
        param_distributions, 
        n_iter=20,  # Number of iterations for RandomizedSearchCV
        cv=5,  # Cross-validation folds
        scoring='neg_mean_squared_error',  # Scoring metric
        random_state=42,  # Ensures reproducibility
        n_jobs=-1  # Use all available cores
    )

    start_time = time.time()
    start_cpu = psutil.cpu_percent(interval=None)
    random_search.fit(X_train, y_train)
    end_cpu = psutil.cpu_percent(interval=None)
    end_time = time.time()

    # Calculate time and CPU usage
    execution_time = end_time - start_time
    avg_cpu_usage = (start_cpu + end_cpu) / 2

    y_pred = random_search.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mape = mean_absolute_percentage_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)

    # Calculate the range of the target variable
    target_range = y_train.max() - y_train.min()

    # Calculate normalized RMSE (nRMSE)
    nrmse = rmse / target_range
    
    memory_usage_MB = X_train.memory_usage(deep=True).sum() / (1024 ** 2)
    normalized_time = execution_time / memory_usage_MB
    
    return {
        'RMSE': rmse,
        'MAPE': mape,
        'R2': r2,
        'MSE': mse,
        'nRMSE': nrmse,  # Normalized RMSE
        'Execution Time (Raw)': execution_time,  # Raw execution time
        'Normalized Time (s/MB)': normalized_time,  # Normalized execution time
        'Average CPU Usage': avg_cpu_usage
    }


In [6]:
# Define sample sizes
sample_sizes = [1.0, 0.5, 0.25, 0.125, 100, 1000, 10000, 100000]

# Initialize metrics storage
metrics_list = []
total_execution_time = 0
total_cpu_usage = 0
total_memory_usage_MB = 0

# Loop through each sample size
for size in sample_sizes:
    try:
        X_train_sample, X_test_sample, y_train_sample, y_test_sample = sample_data(X, y, size)
        metrics = calculate_metrics(X_train_sample, X_test_sample, y_train_sample, y_test_sample)
        metrics['Sample Size'] = size
        metrics_list.append(metrics)

        # Accumulate total       
        total_execution_time += metrics['Execution Time (Raw)']
        total_cpu_usage += metrics['Average CPU Usage']
        total_memory_usage_MB += X_train_sample.memory_usage(deep=True).sum() / (1024 ** 2)

        print(f"Metrics for sample size {size}:")
        for key, value in metrics.items():
            print(f"{key}: {value}")
        print("-" * 50)

    except Exception as e:
        print(f"An error occurred for sample size {size}: {e}")


Metrics for sample size 1.0:
RMSE: 1.302020618484002
MAPE: 0.17614821110550927
R2: 0.3900112500044761
MSE: 1.695257690957463
nRMSE: 0.14466895760933354
Execution Time (Raw): 680.6110570430756
Normalized Time (s/MB): 1.6477909838656073
Average CPU Usage: 61.1
Sample Size: 1.0
--------------------------------------------------
Metrics for sample size 0.5:
RMSE: 1.2934977989938767
MAPE: 0.17550371297482062
R2: 0.3975680286627875
MSE: 1.6731365560020037
nRMSE: 0.1437219776659863
Execution Time (Raw): 344.0713667869568
Normalized Time (s/MB): 1.6643596127283566
Average CPU Usage: 54.800000000000004
Sample Size: 0.5
--------------------------------------------------
Metrics for sample size 0.25:
RMSE: 1.2936233293775434
MAPE: 0.1755121274588317
R2: 0.3974510942759174
MSE: 1.6734613183098404
nRMSE: 0.14373592548639372
Execution Time (Raw): 172.06752276420593
Normalized Time (s/MB): 1.6646681957090048
Average CPU Usage: 53.849999999999994
Sample Size: 0.25
-------------------------------------

In [7]:
# Convert metrics to DataFrame
metrics_df = pd.DataFrame(metrics_list)

# Calculate total metrics
total_avg_cpu_usage = total_cpu_usage / len(sample_sizes)
normalized_total_time = total_execution_time / total_memory_usage_MB

# Convert total execution time to minutes and seconds
total_minutes = int(total_execution_time // 60)
total_seconds = total_execution_time % 60

# Display total metrics
print(f"Total Execution Time for Entire Process (Raw): {total_minutes} minutes and {total_seconds:.2f} seconds")
print(f"Total Normalized Execution Time for Entire Process: {normalized_total_time:.8f} seconds per MB")
print(f"Total Average CPU Usage for Entire Process: {total_avg_cpu_usage:.2f}%")

# Display the metrics DataFrame
metrics_df


Total Execution Time for Entire Process (Raw): 21 minutes and 25.63 seconds
Total Normalized Execution Time for Entire Process: 1.64630008 seconds per MB
Total Average CPU Usage for Entire Process: 45.71%


,RMSE,MAPE,R2,MSE,nRMSE,Execution Time (Raw),Normalized Time (s/MB),Average CPU Usage,Sample Size
0,1.302021,0.176148,0.390011,1.695258,0.144669,680.611057,1.647791,61.10,1.000
1,1.293498,0.175504,0.397568,1.673137,0.143722,344.071367,1.664360,54.80,0.500
2,1.293623,0.175512,0.397451,1.673461,0.143736,172.067523,1.664668,53.85,0.250
3,1.294206,0.175588,0.396908,1.674968,0.143801,85.708589,1.658376,54.20,0.125
4,1.651158,0.251373,0.374697,2.726321,0.206395,0.117211,21.337630,31.40,100.000
5,1.273833,0.148232,0.404395,1.622651,0.141537,0.118210,2.151940,31.10,1000.000
6,1.353992,0.188068,0.381729,1.833294,0.150444,0.194056,0.353268,35.25,10000.000
7,1.291878,0.175202,0.401621,1.668949,0.143542,2.745755,0.499850,43.95,100000.000
